In [1]:
-

In [2]:

from pyspark.sql import SparkSession 
from pyspark.sql import DataFrame 
from pyspark.sql import Column 
from pyspark.sql import Row 
from pyspark.sql import GroupedData
from pyspark.sql import DataFrameNaFunctions 
from pyspark.sql import DataFrameStatFunctions
from pyspark.sql import functions 
from pyspark.sql import types 
from pyspark.sql import Window 
from pyspark.sql import HiveContext
from pyspark.sql import SQLContext



In [3]:
spark = SparkSession.builder.appName("sqlchtshit").master('local[2]').config(conf=conf).getOrCreate()

In [4]:
df = spark.createDataFrame([
        ("Geoffrey", "sagini", 29, +254715245874),
        ("Duke", "Mogambi", 28, +254717730881),
        ("Shadrack", "Osoro", 18, +2543726244461),
        ("James", "Nyabicha", 21, +254706456765),
        ("Margrate" ,"Nyabicha", 45, +254726244461)
    ], ["firstName", "lastName", "age", "phoneNumber"])

In [12]:
df.select(df.firstName).show()

+---------+
|firstName|
+---------+
| Geoffrey|
|     Duke|
| Shadrack|
|    James|
| Margrate|
+---------+



In [16]:
from pyspark.sql.functions import col

In [17]:
df.select(col("firstName")).show()

+---------+
|firstName|
+---------+
| Geoffrey|
|     Duke|
| Shadrack|
|    James|
| Margrate|
+---------+



In [18]:
df.select("firstName").show()

+---------+
|firstName|
+---------+
| Geoffrey|
|     Duke|
| Shadrack|
|    James|
| Margrate|
+---------+



In [19]:
df.select(df["firstName"]).show()

+---------+
|firstName|
+---------+
| Geoffrey|
|     Duke|
| Shadrack|
|    James|
| Margrate|
+---------+



In [21]:
from pyspark.sql.functions import explode

In [ ]:
df.select("firstName", "age", explode("phoneNumber")\
         .alias("contactInfo"))\
         .select("contactInfo.type","firstName","age").show()  

In [24]:
#using when
from pyspark.sql import functions as F
df.select("firstName", F.when(df.age > 30, 1).otherwise(0)).show()

+---------+--------------------------------------+
|firstName|CASE WHEN (age > 30) THEN 1 ELSE 0 END|
+---------+--------------------------------------+
| Geoffrey|                                     0|
|     Duke|                                     0|
| Shadrack|                                     0|
|    James|                                     0|
| Margrate|                                     1|
+---------+--------------------------------------+



In [25]:
df[df.firstName.isin("Geoffrey", "Sagini")].collect()

[Row(firstName='Geoffrey', lastName='sagini', age=29, phoneNumber=254715245874)]

In [26]:
#like
df.select("firstName", df.lastName.like("cha")).show()


+---------+-----------------+
|firstName|lastName LIKE cha|
+---------+-----------------+
| Geoffrey|            false|
|     Duke|            false|
| Shadrack|            false|
|    James|            false|
| Margrate|            false|
+---------+-----------------+



In [27]:
#startswith and endswith
df.select("firstName", df.lastName.startswith("sa")).show()

+---------+------------------------+
|firstName|startswith(lastName, sa)|
+---------+------------------------+
| Geoffrey|                    true|
|     Duke|                   false|
| Shadrack|                   false|
|    James|                   false|
| Margrate|                   false|
+---------+------------------------+



In [28]:
df.select(df.lastName.endswith("a")).show()

+---------------------+
|endswith(lastName, a)|
+---------------------+
|                false|
|                false|
|                false|
|                 true|
|                 true|
+---------------------+



In [30]:
#substring
df.select(df.firstName.substr(1,3).alias("name"))\
          .collect()

[Row(name='Geo'),
 Row(name='Duk'),
 Row(name='Sha'),
 Row(name='Jam'),
 Row(name='Mar')]

In [31]:
#between
df.select(df.age.between(22,24)).show()

+-----------------------------+
|((age >= 22) AND (age <= 24))|
+-----------------------------+
|                        false|
|                        false|
|                        false|
|                        false|
|                        false|
+-----------------------------+



In [40]:
#adding, update and remove columns
df = df.withColumn('postalcode', df.firstName)\
       .withColumn('city', df.age)\
       .withColumn('state', df.age)\
       .withColumn('streetaddress', df.phoneNumber)
    
    
       

In [35]:
df.show()

+---------+--------+---+-------------+--------+---+-------------+
|firstName|lastname|age|  phoneNumber|     Tom| 40|  +2547654677|
+---------+--------+---+-------------+--------+---+-------------+
| Geoffrey|  sagini| 29| 254715245874|Geoffrey| 29| 254715245874|
|     Duke| Mogambi| 28| 254717730881|    Duke| 28| 254717730881|
| Shadrack|   Osoro| 18|2543726244461|Shadrack| 18|2543726244461|
|    James|Nyabicha| 21| 254706456765|   James| 21| 254706456765|
| Margrate|Nyabicha| 45| 254726244461|Margrate| 45| 254726244461|
+---------+--------+---+-------------+--------+---+-------------+



In [38]:
df.select("lastName").show()

+--------+
|lastName|
+--------+
|  sagini|
| Mogambi|
|   Osoro|
|Nyabicha|
|Nyabicha|
+--------+



In [41]:
df.show()

+---------+--------+---+-------------+--------+---+-------------+----------+----+-----+-------------+
|firstName|lastname|age|  phoneNumber|     Tom| 40|  +2547654677|postalcode|city|state|streetaddress|
+---------+--------+---+-------------+--------+---+-------------+----------+----+-----+-------------+
| Geoffrey|  sagini| 29| 254715245874|Geoffrey| 29| 254715245874|  Geoffrey|  29|   29| 254715245874|
|     Duke| Mogambi| 28| 254717730881|    Duke| 28| 254717730881|      Duke|  28|   28| 254717730881|
| Shadrack|   Osoro| 18|2543726244461|Shadrack| 18|2543726244461|  Shadrack|  18|   18|2543726244461|
|    James|Nyabicha| 21| 254706456765|   James| 21| 254706456765|     James|  21|   21| 254706456765|
| Margrate|Nyabicha| 45| 254726244461|Margrate| 45| 254726244461|  Margrate|  45|   45| 254726244461|
+---------+--------+---+-------------+--------+---+-------------+----------+----+-----+-------------+



In [43]:
df.withColumnRenamed("40", "forty").select("forty").show()

+-----+
|forty|
+-----+
|   29|
|   28|
|   18|
|   21|
|   45|
+-----+



In [47]:
df.drop("Tom", "forty").show()

+---------+--------+---+-------------+---+-------------+----------+----+-----+-------------+
|firstName|lastname|age|  phoneNumber| 40|  +2547654677|postalcode|city|state|streetaddress|
+---------+--------+---+-------------+---+-------------+----------+----+-----+-------------+
| Geoffrey|  sagini| 29| 254715245874| 29| 254715245874|  Geoffrey|  29|   29| 254715245874|
|     Duke| Mogambi| 28| 254717730881| 28| 254717730881|      Duke|  28|   28| 254717730881|
| Shadrack|   Osoro| 18|2543726244461| 18|2543726244461|  Shadrack|  18|   18|2543726244461|
|    James|Nyabicha| 21| 254706456765| 21| 254706456765|     James|  21|   21| 254706456765|
| Margrate|Nyabicha| 45| 254726244461| 45| 254726244461|  Margrate|  45|   45| 254726244461|
+---------+--------+---+-------------+---+-------------+----------+----+-----+-------------+



In [48]:
#sorting the above dataframe
df.sort("age", ascending = False).collect()

[Row(firstName='Margrate', lastname='Nyabicha', age=45, phoneNumber=254726244461, Tom='Margrate', 40=45, +2547654677=254726244461, postalcode='Margrate', city=45, state=45, streetaddress=254726244461),
 Row(firstName='Geoffrey', lastname='sagini', age=29, phoneNumber=254715245874, Tom='Geoffrey', 40=29, +2547654677=254715245874, postalcode='Geoffrey', city=29, state=29, streetaddress=254715245874),
 Row(firstName='Duke', lastname='Mogambi', age=28, phoneNumber=254717730881, Tom='Duke', 40=28, +2547654677=254717730881, postalcode='Duke', city=28, state=28, streetaddress=254717730881),
 Row(firstName='James', lastname='Nyabicha', age=21, phoneNumber=254706456765, Tom='James', 40=21, +2547654677=254706456765, postalcode='James', city=21, state=21, streetaddress=254706456765),
 Row(firstName='Shadrack', lastname='Osoro', age=18, phoneNumber=2543726244461, Tom='Shadrack', 40=18, +2547654677=2543726244461, postalcode='Shadrack', city=18, state=18, streetaddress=2543726244461)]

In [49]:
df.columns

['firstName',
 'lastname',
 'age',
 'phoneNumber',
 'Tom',
 '40',
 '+2547654677',
 'postalcode',
 'city',
 'state',
 'streetaddress']

In [51]:
df.repartition(4).rdd.getNumPartitions()

4

In [52]:
df.coalesce(2).rdd.getNumPartitions()

2

In [ ]:
df.createTempView("people")
df.createOrReplaceTempView("people1")
df.createGlobalTempView("people2")
df.createOrReplaceGlobalTempView("people2")

In [ ]:
spark.sql("select * from people").show()
spark.sql("select * from global_temp.people1").show()

In [ ]:
#data structire output
rdd = df.rdd
df.toJson()
df.toPandas()

In [ ]:
df.select("firstName", "lastName)", "age")\
          .write\
          .save("C:\sparkfiles\nameandcity.json", format="json")